### Imports and loading config data

In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np
import yaml
import os
from sklearn import mixture
from collections import defaultdict
from scipy.spatial import distance
from scipy.cluster.hierarchy import dendrogram
from scipy.stats import mannwhitneyu
import scipy.cluster.hierarchy as hc
import scipy.spatial as sp
import itertools
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Load configuration file
with open('params_icgcl.yaml', 'rb') as f:
    conf = yaml.safe_load(f.read())   

## Plot settings
sns.set_style(style='white')

plt.rc('text', usetex = True)
plt.rc('font', **{'family' : "sans-serif"})
plt.rc('text.latex', preamble=r"\usepackage{amsmath}"
           r"\usepackage{amstext}")
plt.rcParams["axes.linewidth"] = 2.50
plt.rcParams['xtick.major.size'] = 20
plt.rcParams['ytick.major.size'] = 20
fsz = 28

In [ ]:
# Load general settings from the config file
save_img = conf['settings']['save_img']
dataset_config = conf["datasets"]["LINE-1"]

# Load dataset-specific parameters
input_file = dataset_config["input_file"]
input_folder = dataset_config["input_folder"]
output_folder = dataset_config["output_folder"]
ec_hc_mask_folder = dataset_config["ec_hc_mask_folder"]
rep_list = dataset_config["rep_list"]
expt_list = [item for sublist in rep_list for item in sublist]

In [ ]:
df_list_treat = {'WRN ASO': 'c','HGPS ASO': '#66CDAA', 'WT': 'g', 'WRN SCR': 'r', 'WRN NT':'#D70040', 'HGPS SCR': 'm','HGPS NT':'#8B008B'}

### Data preprocessing

In [ ]:
def data_preprocessing(df,expt_list):
    normalized_df = df[['Gene_ID','StartPos','EndPos']]
    for expt in expt_list:
        normalized_df[expt] = (df[expt])/  df[expt].sum()
    normalized_df = normalized_df.reset_index()
    return  normalized_df

### Visualization Functions

In [ ]:
def setAlpha(ax,a):
    for art in ax.get_children():
        if isinstance(art, PolyCollection):
            art.set_alpha(a)

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    fig, ax = plt.subplots()
    fig.set_size_inches(9,6)
    fig.set_dpi(300)
    df_list_treat = {'WRN ASO': 'c','HGPS ASO': '#66CDAA', 'WT': 'g', 'WRN SCR': 'r', 'WRN NT':'#D70040', 'HGPS SCR': 'm','HGPS NT':'#8B008B'}
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    ddata = dendrogram(*args, **kwargs, color_threshold=0, above_threshold_color='k') # Remove the color_threshold and above_threshold_color to get colors for the dendrogram lines
    if not kwargs.get('no_plot', False):
        plt.xlabel("Experiment",fontsize = fsz, color='k')
        plt.ylabel("Distance",fontsize = fsz, color='k')
        plt.tick_params(labelsize=fsz-6)
        plt.tick_params(labelsize=fsz-6)
        plt.tick_params(axis=u'both', which=u'both',length=0)
        if max_d:
            plt.axhline(y=max_d, c='k')
    # Add colors to the dendrogram
    ax = plt.gca()
    xlbls = ax.get_xmajorticklabels()
    for lbl in xlbls:
        old_txt = lbl.get_text() # Way to text color of axis of cluster map
        temp = [i for i, s in enumerate(df_list_treat) if all([x in old_txt for x in s.split(' ')])]
        lbl.set_color(list(df_list_treat.items())[int(temp[0])][1])
    ax.grid(False)
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    plt.savefig(os.path.join(output_folder,'dendrogram_SVD20.pdf'), format='pdf', bbox_inches='tight')
    return ddata

In [ ]:
# Clustering plot using seaborn based on the disease
def plot_save_dendrogram_PCA_SVD(output_folder):
    input_file = os.path.join(output_folder,'fpkm_data_SVD20.csv') # Use fpkm_data_SVD.csv or fpkm_data_PCA.csv
    df_sort = pd.read_csv(input_file)
    df = df_sort.sort_values('chromosome_id')
    row_labels = list(df.columns[df.columns != 'chromosome_id'])
    converter = lambda x: x.replace('_', '-')
    row_labels = list(map(converter, row_labels))
    row_labels = ['-'.join(x.split('-')[:-1]) for x in row_labels]

    df_list = df.loc[:, df.columns != 'chromosome_id'].to_numpy()
    Z = hc.linkage(sp.distance.squareform(df_list), method='average', metric='cityblock', optimal_ordering=True)
    fancy_dendrogram(
    Z,
    labels=row_labels,
    p=20,
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True
    )

In [ ]:
# Setting grid labels colors based on the treatment (or disease)
def set_cluster_grid_labels_color(df_list_treat,cluster_grid, col_labels, row_labels):
    old_lbls_txt_obj = cluster_grid.ax_heatmap.xaxis.get_majorticklabels()
    new_col_lbls = []
    new_col_clr = []
    for obj in old_lbls_txt_obj:
        old_txt = obj.get_text() # Way to text color of axis of cluster map (
        idx = int(old_txt)
        obj.set_text(col_labels[idx])
        new_col_lbls.append(obj)
        temp = [i for i, s in enumerate(df_list_treat) if all([x in col_labels[idx] for x in s.split(' ')])]
        obj.set_color(list(df_list_treat.items())[int(temp[0])][1])

    old_lbls_txt_obj = cluster_grid.ax_heatmap.yaxis.get_majorticklabels()
    new_row_lbls = []
    for obj in old_lbls_txt_obj:
        old_txt = obj.get_text()
        idx = int(old_txt)
        obj.set_text(row_labels[idx])
        new_row_lbls.append(obj)
    cluster_grid.ax_heatmap.set_xticklabels(new_col_lbls, rotation=90, fontsize=fsz-6)
    cluster_grid.ax_heatmap.set_yticklabels(new_row_lbls, rotation=0, fontsize=fsz-6)

In [ ]:
# Clustering plot using seaborn based on the phenotype
def plot_save_dendrogram_ellstar(output_folder):
    input_file = os.path.join(output_folder,'auto_cor_ratio.csv')
    df_sort = pd.read_csv(input_file)
    df = df_sort.sort_values('chromosome_id')
    col_labels = list(df.columns[df.columns != 'chromosome_id'])
    converter = lambda x: x.replace('_', '-')
    col_labels = list(map(converter, col_labels))
    col_labels = ['-'.join(x.split('-')[:-1]) for x in col_labels]
    row_labels = df['chromosome_id'].tolist()
    df_list = df.loc[:, df.columns != 'chromosome_id'].to_numpy()
    kws = dict(figsize=(8, 7))
    cluster_grid = sns.clustermap(df_list, cmap='mako',method='average',metric='cityblock',  **kws)
    cluster_grid.ax_cbar.tick_params(axis='y', length=2)
    for spine in cluster_grid.ax_cbar.spines:
        cluster_grid.ax_cbar.spines[spine].set_color('black')
        cluster_grid.ax_cbar.spines[spine].set_linewidth(2)

    ax = cluster_grid.ax_heatmap
    ax.figure.axes[-1].set_ylabel(r'$\ell^{*}$', size=fsz)
    ax.figure.axes[-1].tick_params(labelsize=fsz-12)
    ax.set_xlabel("Experiment",fontsize = fsz, color='k')
    ax.set_ylabel("Chromosome",fontsize = fsz, color='k')
    ax.tick_params(axis='y', length=1)
    ax.tick_params(axis='x', length=1)
    set_cluster_grid_labels_color(df_list_treat,cluster_grid, col_labels, row_labels)
    # Drawing the frame
    for a in cluster_grid.ax_row_dendrogram.collections:
        a.set_linewidth(1.2)

    for a in cluster_grid.ax_col_dendrogram.collections:
        a.set_linewidth(1.2)
    for _, spine in ax.spines.items():
        spine.set_visible(True)
        spine.set_linewidth(3)
        spine.set_color("black")
    plt.savefig(os.path.join(output_folder,'dendrogram_ellstar.pdf'), format='pdf', bbox_inches='tight')

### Post Processing data analysis

In [ ]:
# Post processing data analysis beyond this point. Importing the output l* file saved in output_folder directory
df = pd.read_csv(os.path.join(output_folder,"auto_cor_ratio.csv"))

In [ ]:
plot_save_dendrogram_ellstar(output_folder)

In [ ]:
plot_save_dendrogram_PCA_SVD(ec_hc_mask_folder)

In [ ]:
## Fig 5 (main text)
labels = []
for rep in rep_list:
    x = rep[0].split('_')
    if len(x) == 3:
        labels.append(x[0])
    elif len(x) == 5:
        labels.append(x[0]+"_"+x[2]+"_"+x[3])
    else:
        labels.append(x[0]+"_"+x[2])
Chr = ['6','1','4'] #
df_to_plot = pd.DataFrame(columns= ['chromosome_id'] + labels)
for chromosome,data in df.groupby('chromosome_id'):
    if chromosome == Chr[0] or chromosome == Chr[1] or chromosome == Chr[2]:
        df_1 = data
        rep_mean = [chromosome]
        rep_std = [chromosome]
        for rep in rep_list:
            rep_mean.append(df_1[rep].to_numpy().mean())
            rep_std.append(df_1[rep].to_numpy().std())
        df_to_plot = df_to_plot.append(pd.DataFrame([rep_mean], columns=['chromosome_id'] + labels), ignore_index=True)
        df_to_plot = df_to_plot.append(pd.DataFrame([rep_std], columns=['chromosome_id'] + labels), ignore_index=True)
ind = np.arange(len(rep_list))
# Prepare for plotting
x_col = ['WT', 'HGPS_L1_ASO', 'HGPS_SCR','HGPS_NT', 'WRN_L1_ASO', 'WRN_SCR', 'WRN_NT']
labels = x_col
df_to_plot = df_to_plot[[col for col in x_col+["chromosome_id"] if col in df_to_plot.columns]]
rep_mean = df_to_plot[df_to_plot['chromosome_id']==Chr[0]][labels].iloc[[0]].values.flatten().tolist()
rep_std = df_to_plot[df_to_plot['chromosome_id']==Chr[0]][labels].iloc[[1]].values.flatten().tolist()

In [ ]:
colors = ['g', '#66CDAA', 'm', '#8B008B', 'c', 'r', '#D70040']
colors1 = ['k', 'k', 'k', 'k', 'k', 'k', 'k']
bar_kwargs = {'color':colors1,'width':0.4,'linewidth':2}
err_kwargs = {'zorder':0,'fmt':'none','linewidth':2,'ecolor':'k'}  #for matplotlib >= v1.4 use 'fmt':'none' instead
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
fig.set_dpi(300)
plt.style.use('classic')

ax.p1 = plt.bar(ind, rep_mean,alpha =0.2, label= 'Chr='+str(Chr[0]), **bar_kwargs)
ax.errs = plt.errorbar(ind, rep_mean, yerr=rep_std, solid_capstyle='projecting', capsize=5, **err_kwargs)
converter = lambda x: x.replace('_', '-')
labels = list(map(converter, labels))
plt.xticks(ind, labels, color='k')
color = []
for col in labels:
    temp = [i for i, s in enumerate(df_list_treat) if all([x in col for x in s.split(' ')])]
    color.append(list(df_list_treat.items())[temp[0]][1])
i = 0
for xtick in ax.get_xticklabels():
    xtick.set_color(color[i])
    i = i + 1
ax.set_ylabel('$\ell^{*}$', fontsize=fsz+12, color='k')
ax.tick_params('both',which='major', length=7,labelsize=fsz)
ax.tick_params('both',which='minor', length=7,labelsize=fsz)
plt.locator_params(axis='y', nbins=4)
plt.xticks(rotation=70)
plt.show()

In [ ]:
df_chromosome_dist = pd.DataFrame(df.chromosome_id)
for rep_entry in rep_list:
    col_name = ''
    print(rep_entry)
    if any('WT' in rep_name for rep_name in rep_entry):
        col_name = 'WT'
    if any(('HGPS' in rep_name and 'NT' in rep_name) for rep_name in rep_entry):
        col_name = 'HGPS-NT'
    if any(('HGPS' in rep_name and 'SCR' in rep_name) for rep_name in rep_entry):
        col_name = 'HGPS-SCR'
    if any(('HGPS' in rep_name and 'ASO' in rep_name) for rep_name in rep_entry):
        col_name = 'HGPS-L1-ASO'
    if any(('WRN' in rep_name and 'NT' in rep_name) for rep_name in rep_entry):
        col_name = 'WRN-NT'
    if any(('WRN' in rep_name and 'SCR' in rep_name) for rep_name in rep_entry):
        col_name = 'WRN-SCR'
    if any(('WRN' in rep_name and 'ASO' in rep_name) for rep_name in rep_entry):
        col_name = 'WRN-L1-ASO'
    df_chromosome_dist[col_name]=df[rep_entry].mean(axis=1)

In [ ]:
expt_order = ['WT', 'HGPS-L1-ASO','HGPS-SCR','HGPS-NT','WRN-L1-ASO','WRN-SCR','WRN-NT']
df_chromosome_dist_melted_df = pd.melt(df_chromosome_dist,id_vars='chromosome_id',value_vars=list(df_chromosome_dist.columns))
df_chromosome_dist_melted_df=df_chromosome_dist_melted_df.rename(columns={'value':'lstar'})

df_chromosome_dist = df_chromosome_dist.set_index('chromosome_id')
df_chromosome_dist_viol = pd.DataFrame(index=df_chromosome_dist.index)
for col in df_chromosome_dist.columns:
    df_chromosome_dist_viol[col] = (df_chromosome_dist[col]).astype(float)

df_chromosome_dist_viol = df_chromosome_dist_viol.reset_index()
df_chromosome_dist_viol_melted_df = pd.melt(df_chromosome_dist_viol,id_vars='chromosome_id',value_vars=expt_order)
df_chromosome_dist_viol_melted_df = df_chromosome_dist_viol_melted_df.rename(columns={'value':'lstar'})

In [ ]:
# Setup for Figure Supp 5 (Fig S5)
fig, ax = plt.subplots()
fig.set_size_inches(12, 10)
fig.set_dpi(300)
# Let's extract HGPS and WRN separately
df_chromosome_dist_viol_HGPS = df_chromosome_dist_viol_melted_df[(df_chromosome_dist_viol_melted_df['variable'] == 'WT') | (df_chromosome_dist_viol_melted_df['variable'] == 'HGPS-L1-ASO') | (df_chromosome_dist_viol_melted_df['variable'] == 'HGPS-SCR') | (df_chromosome_dist_viol_melted_df['variable'] == 'HGPS-NT')]
df_chromosome_dist_viol_WRN = df_chromosome_dist_viol_melted_df[(df_chromosome_dist_viol_melted_df['variable'] == 'WT') | (df_chromosome_dist_viol_melted_df['variable'] == 'WRN-L1-ASO') | (df_chromosome_dist_viol_melted_df['variable'] == 'WRN-SCR') | (df_chromosome_dist_viol_melted_df['variable'] == 'WRN-NT')]
# To plot Fig S5a
ax = sns.boxplot(data=df_chromosome_dist_viol_WRN, x='variable', y="lstar",palette=['g','c','r','#D70040'], linewidth=2, medianprops=dict(linestyle='None'),meanline=True, showmeans=True, meanprops=dict(linestyle='-',linewidth = 2, color='black'), boxprops=dict(alpha=.6),zorder=1)
# To plot Fig S5 b
# ax = sns.boxplot(data=df_chromosome_dist_viol_HGPS, x='variable', y="lstar",palette=['g','#66CDAA','m','#8B008B'], linewidth=2, medianprops=dict(linestyle='None'),meanline=True, showmeans=True, meanprops=dict(linestyle='-',linewidth = 2, color='black'), boxprops=dict(alpha=.6),zorder=1)
setAlpha(ax,0.6)
ax.grid(False)
ax.set_ylabel('$\ell^{*}$', fontsize=fsz+10, color='k')
ax.set(xlabel=None)
ax.tick_params('both',which='major', length=7,labelsize=fsz)
ax.tick_params('both',which='minor', length=7,labelsize=fsz)
plt.xticks(rotation=45)
plt.show()
plt.tight_layout()
# plt.savefig(os.path.join(output_folder,("violin_plot_ell_L1_dbscan_HGPS"+".pdf")), bbox_inches='tight')

In [ ]:
# Calculate Mann-Whitney test for a sample with respect to a reference.
ref = 'WRN-L1-ASO'
final = 'WRN-NT'
df_ref = df_chromosome_dist_viol_melted_df[df_chromosome_dist_viol_melted_df['variable'] == ref]
df_final = df_chromosome_dist_viol_melted_df[df_chromosome_dist_viol_melted_df['variable'] == final]
x = df_ref['lstar']
y = df_final['lstar']
U,p_val = mannwhitneyu(x, y, alternative='two-sided')
# No significant difference were seen using Mann Whitney test

In [ ]:
# Reading the files containing HAT and LAT genes. The genes labeled 1 are HATs and genes labeled 0 are LATs
df_ec_hc = pd.DataFrame()
counter = 0
for chromosome_dir in os.listdir(ec_hc_mask_folder):
    f = os.path.join(ec_hc_mask_folder,chromosome_dir)
    if os.path.isdir(f):
        for filename in os.listdir(f):
            if filename.endswith("_ec_hc_mask_xy.csv"):  # Check for files that match the pattern
                file_path = os.path.join(f, filename)  # Get the full path of the file
                df_ec_hc_temp = pd.read_csv(file_path)
                df_ec_hc_temp.drop(['StartPos','EndPos'],axis=1,inplace=True)
                df_ec_hc_temp = df_ec_hc_temp.set_index('Gene_ID')
                if counter == 0:
                    df_ec_hc = df_ec_hc_temp
                else:
                    df_ec_hc = df_ec_hc.add(df_ec_hc_temp, fill_value=0)
                counter = counter+1
df_ec_hc['sum'] = df_ec_hc.sum(axis=1)
df_ec_hc = df_ec_hc[df_ec_hc['sum']>0]
index_ec_genes = list(df_ec_hc.index)

### Analysis and plotting for Fig. 3
### Pairwise similarity plot. 

In [ ]:
## Analysis and plotting for Fig. 3
# Pairwise similarity plot. 
# First we will calculate the manhattan distance for l* with respect to a reference (WT in this case)
df.sort_values(by=['chromosome_id'],inplace=True)
df_vec = df.reset_index(drop=True)
df_list = df_vec.loc[:, df.columns != 'chromosome_id'].to_numpy()
dist_df = pd.DataFrame(columns= ['reference'] + expt_list)
ref_list = rep_list[0] # WT is used as a reference
# Let's assign the ref_list for this case
label_list = []
for item in rep_list:
    x = item[0].split('_')
    if len(x) == 3:
        label_list.append(x[0])
    elif len(x) == 5:
        label_list.append(x[0]+"_"+x[2]+"_"+x[3])
    else:
        label_list.append(x[0]+"_"+x[2])
dist_df['reference'] = ref_list
print(df_vec)
for i in range(0,len(rep_list)):
    for rep1, rep2 in itertools.product(ref_list,rep_list[i]):
        dist = distance.cityblock(df_vec[rep1],df_vec[rep2])
        dist_df.loc[dist_df['reference'] == rep1,rep2] = dist
print(dist_df)

In [ ]:
# Setting up the mean and standard deviation plot
dist_list = []
dist_list_std = []
dist_list_df = pd.DataFrame(columns= label_list)
dist_list_df_std = pd.DataFrame(columns= label_list)
for rep in rep_list:
    dist_list.append(np.average(dist_df[rep].mean().to_numpy()))
    dist_list_std.append(np.std(dist_df[rep].to_numpy()))

dist_list_df = dist_list_df.append(pd.DataFrame([dist_list], columns= label_list), ignore_index=True)
dist_list_df_std = dist_list_df_std.append(pd.DataFrame([dist_list_std], columns= label_list), ignore_index=True)
# Setting up the plotting tool
df_sim_auto_corr = pd.concat([dist_list_df,dist_list_df_std])
df_sim_auto_corr = df_sim_auto_corr.T.reset_index()
df_sim_auto_corr.columns = ['Expt','mean','std']
x_col = ['WT', 'HGPS_L1_ASO', 'HGPS_SCR','HGPS_NT', 'WRN_L1_ASO', 'WRN_SCR', 'WRN_NT']
df_sim_auto_corr['Expt'] = pd.Categorical(df_sim_auto_corr.Expt, categories=x_col, ordered=True)
df_sim_auto_corr = df_sim_auto_corr.sort_values('Expt')
print(df_sim_auto_corr)
# Save these values as well
# df_main.to_csv(os.path.join(output_folder,"pair_wise_similarity_autocorr_L1_dbscan_gamma1.csv"),index=False)


In [ ]:
# Calculating the manhattan distance for the raw expression data with respect to a reference (WT in this case)
df = pd.read_csv(input_file)
result = defaultdict(list)
dist_list = []
dist_list_std = []
dist_list_df = pd.DataFrame(columns= label_list)

for i in range(0,len(rep_list)):
    result = []
    for rep1, rep2 in itertools.product(ref_list,rep_list[i]):
        dist = 0
        counter = 0
        for chromosome,data in df.groupby('Chromosome'):
            # if chromosome.isdigit() and 0 < int(chromosome) < 23:
            if chromosome.isalnum() and chromosome != 'MT':
                counter = counter + 1
                normalized_df = data_preprocessing(data,[rep1,rep2])
                normalized_df = normalized_df[normalized_df['Gene_ID'].isin(index_ec_genes)].reset_index(drop=True) # Filter out EC genes
                # Let's calculate the similarity distance between the two distributions
                dist = dist + distance.cityblock(normalized_df[rep1],normalized_df[rep2])
        avg_dist = dist/counter
        result.append(avg_dist)
    dist_list_df[label_list[i]] = result
    
for rep in label_list:
    dist_list.append(dist_list_df[rep].mean()) # Take mean across chromosome
    dist_list_std.append(dist_list_df[rep].std())
df_sim_exp = pd.DataFrame([dist_list,dist_list_std],columns = label_list)
df_sim_exp = df_sim_exp.T.reset_index()
df_sim_exp.columns = ['Expt','mean','std']
x_col = ['WT', 'HGPS_L1_ASO', 'HGPS_SCR','HGPS_NT', 'WRN_L1_ASO', 'WRN_SCR', 'WRN_NT']
df_sim_exp['Expt'] = pd.Categorical(df_sim_exp.Expt, categories=x_col, ordered=True)
df_sim_exp.sort_values('Expt', inplace=True)
print(df_sim_exp)

# Let's save these values as well
# df_main.to_csv(os.path.join(output_folder,"pair_wise_similarity_expression_L1_dbscan_gamma1.csv"),index=False)

In [ ]:
## We can plot now
y_exp = df_sim_exp['mean'].to_list()
yerr_exp = df_sim_exp['std'].to_list()
y_autocorr = df_sim_auto_corr['mean'].to_list()
yerr_autocorr = df_sim_auto_corr['std'].to_list()
x_label = df_sim_auto_corr['Expt'].to_list()

y_exp.insert(4, y_exp[0])
yerr_exp.insert(4, yerr_exp[0])
y_autocorr.insert(4, y_autocorr[0])
yerr_autocorr.insert(4, yerr_autocorr[0])
x_label.insert(4, x_label[0])
x1 = [1,2,3,4]

# Setting up the plotting tool
fsz = 28
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
fig.set_dpi(300)

r_list = (0,4) # 0,4 for HGPS; 4,8 for WRN
plt.plot(x1,y_exp[r_list[0]:r_list[1]],'-b',linewidth = 3)
plt.errorbar(x1, y_exp[r_list[0]:r_list[1]], yerr=yerr_exp[r_list[0]:r_list[1]], fmt='o', markersize=8, capsize=10,markeredgecolor='b', color="b", elinewidth=2, markeredgewidth=2)
plt.xticks(rotation=45)
ax1 = ax.twinx()
ax1.plot(x1,y_autocorr[r_list[0]:r_list[1]],'-r',linewidth = 3)
ax1.errorbar(x1, y_autocorr[r_list[0]:r_list[1]], yerr=yerr_autocorr[r_list[0]:r_list[1]], fmt='o', markersize=8,markeredgecolor='r', capsize=10, color="r", elinewidth=2, markeredgewidth=2)

ax.set_ylabel(r'$\mathcal{D}(\alpha_t)$', fontsize=fsz+10, color='k')
ax1.set_ylabel(r'$\mathcal{D}(\ell^{*})$', fontsize=fsz+10, color='k')
ax.set_xlim(left=0.9,right=4.1)
ax1.set_ylim(bottom=0,top=50)
ax.set_ylim(bottom=0,top=0.5)

ax1.locator_params(axis = 'y', nbins=5)
ax.locator_params(axis = 'y', nbins=5)
converter = lambda x: x.replace('_', '-')
x_label = list(map(converter, x_label))

ax.set_xticks(x1)
ax.set_xticklabels(x_label[r_list[0]:r_list[1]])

# Setting the color for the axis
color = []
for col in x_label[r_list[0]:r_list[1]]:
    # temp = [i for i, s in enumerate(df_list_treat) if s in col]
    temp = [i for i, s in enumerate(df_list_treat) if all([x in col for x in s.split(' ')])]
    color.append(list(df_list_treat.items())[temp[0]][1])
i = 0
for xtick in ax.get_xticklabels():
    xtick.set_color(color[i])
    i = i + 1

ax.yaxis.label.set_color('b')
ax1.spines["left"].set_edgecolor('b')
ax.tick_params(axis='y', colors='b')
ax1.yaxis.label.set_color('r')
ax1.spines["right"].set_edgecolor('r')
ax1.tick_params(axis='y', colors='r')
ax.tick_params('both',which='major', length=7,labelsize=fsz)
ax.tick_params('both',which='minor', length=7,labelsize=fsz)
ax1.tick_params('both',which='major', length=7,labelsize=fsz)
ax1.tick_params('both',which='minor', length=7,labelsize=fsz)
plt.tight_layout()
# plt.savefig(os.path.join(output_folder,("pair_wise_similarity_HGPS_dbscan_gamma1"+".pdf")), bbox_inches='tight')

### Differential analysis plot

In [ ]:
## Let's work on the differential analysis plot
# Fig S6 is done
counts_df = pd.read_csv(os.path.join(input_folder,"Line1","DiffAnalysis","Line1_Deseq2.csv"))
clinical_df = pd.read_csv(os.path.join(input_folder,"Line1","DiffAnalysis","Input_type.csv"))
input_file_genelen = pd.read_csv(os.path.join(input_folder,"Line1","DiffAnalysis","Gene_length.csv"))
counts_df = counts_df.set_index('Gene_ID')
clinical_df = clinical_df.set_index('Samples')
counts_df = counts_df.T

In [ ]:
# Preprocessing the data for which annotations are missing and exclude genes with very low levels of expression
samples_to_keep = ~clinical_df.condition.isna()
counts_df = counts_df.loc[samples_to_keep]
clinical_df = clinical_df.loc[samples_to_keep]

In [ ]:
# filter out genes that have less than 10 read counts
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

In [ ]:
dds = DeseqDataSet(
    counts=counts_df,
    metadata=clinical_df,
    design_factors="condition",
    refit_cooks=True,
    ref_level = ["condition", "WT"], #Here we consider the reference level to be the WT samples
    n_cpus=4,
)

dds.deseq2()

In [ ]:
stat_res = DeseqStats(dds, contrast=['condition', 'HGPS_NT', 'WT'], n_cpus=4)  
# Change 'HGPS_NT' to other conditions if needed

results_summary = stat_res.summary()
stat_res.lfc_shrink(coeff="condition_HGPS_NT_vs_WT")

In [ ]:
stat_res_padj = stat_res.results_df[stat_res.results_df['padj']<1e-4]
up_regulated = stat_res_padj[stat_res_padj['log2FoldChange'] > 0.0]
down_regulated = stat_res_padj[stat_res_padj['log2FoldChange'] < 0.0]
stat_res_padj = stat_res_padj.reset_index()

In [ ]:
df_annot = pd.read_csv(os.path.join(input_folder,"Line1","DiffAnalysis","GeneId_Start_End_Chr.csv"))

lfc_mark = stat_res_padj[(stat_res_padj['log2FoldChange'] < 0) | (stat_res_padj['log2FoldChange'] > 0)][['Gene_ID','log2FoldChange','padj']]
# Let's find the genes on chromosome 1 and 6
df2 = pd.merge(df_annot, lfc_mark, on = "Gene_ID", how = 'inner')
df2.sort_values("log2FoldChange",inplace = True, ascending = False)
df2 = pd.concat([df2.head(100),df2.tail(100)])
print(df2)

In [ ]:
lfc_neg1 = stat_res_padj[stat_res_padj['log2FoldChange'] < -1]
lfc_pos1 = stat_res_padj[stat_res_padj['log2FoldChange'] > 1]
lfc_thr1 = stat_res_padj[stat_res_padj['log2FoldChange'].between(-1, 1)]

lfc_neg = df2[df2['log2FoldChange'] < -1]
lfc_pos = df2[df2['log2FoldChange'] > 1]
lfc_thr = df2[df2['log2FoldChange'].between(-1, 1)]

In [ ]:
df2_up_Chr1 = df2[df2['Chromosome'] == '1'].reset_index()
df2_up_Chr6 = df2[df2['Chromosome'] == '6'].reset_index()

x_Chr6, y_Chr6, s_Chr6 = df2_up_Chr6['log2FoldChange'], df2_up_Chr6['padj'], df2_up_Chr6['Chromosome']
x_Chr1, y_Chr1, s_Chr1 = df2_up_Chr1['log2FoldChange'], df2_up_Chr1['padj'], df2_up_Chr1['Chromosome']

In [ ]:
x_gene = df2[df2['Gene_ID'].isin(['ENSG00000117595', 'ENSG00000162433'])]['log2FoldChange']
y_gene = df2[df2['Gene_ID'].isin(['ENSG00000117595', 'ENSG00000162433'])]['padj']
s_gene = ['AK4', 'IRF6']

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(12, 8)
fig.set_dpi(300)

al_val = 0.05
plt.plot(lfc_neg1['log2FoldChange'], -np.log10(lfc_neg1['padj']), 'or', alpha=al_val, markersize=6)
plt.plot(lfc_pos1['log2FoldChange'], -np.log10(lfc_pos1['padj']), 'og', alpha=al_val, markersize=6)
plt.plot(lfc_thr1['log2FoldChange'], -np.log10(lfc_thr1['padj']), 'o', color='grey', alpha=al_val, markersize=6)

al_val = 0.8
plt.plot(lfc_neg['log2FoldChange'], -np.log10(lfc_neg['padj']), 'or', alpha=al_val, markersize=6)
plt.plot(lfc_pos['log2FoldChange'], -np.log10(lfc_pos['padj']), 'og', alpha=al_val, markersize=6)
plt.plot(lfc_thr['log2FoldChange'], -np.log10(lfc_thr['padj']), 'o', color='grey', alpha=al_val, markersize=6)

for i, txt in enumerate(s_Chr6):
    ax.annotate(txt, (x_Chr6.iloc[i], -np.log10(y_Chr6.iloc[i])), fontsize=fsz-10)

ax.set_ylabel('$-\log_{10}$(P-value)', fontsize=fsz, color='k')
ax.set_xlabel('$\log_2$(Fold change)', fontsize=fsz, color='k')
ax.tick_params('both', which='major', length=7, labelsize=fsz)
ax.tick_params('both', which='minor', length=7, labelsize=fsz)

# plt.savefig(os.path.join(output_folder, "Diff_Analysis_Chr6_HGPS_NT_WT.pdf"), bbox_inches='tight')
plt.show()


### Creating Subplots

In [ ]:
## Fig S2 a and b
def fit_distribution_2d_comp(normalized_df, expt_name):
    fig, axs = plt.subplots(1, len(expt_name), figsize=(14, 6), sharex=True, sharey=True)
    i = 0
    # Plot PDF as a histogram
    title_label = ["WT", "HGPS-NT"]
    for expt in expt_name:
        print(expt)
        m6_gid = mask_6.loc[mask_6[expt] == 1]['Gene_ID'].tolist()
        temp = normalized_df.loc[normalized_df.Gene_ID.isin(m6_gid)]
        dpgmm = mixture.BayesianGaussianMixture(n_components=10, covariance_type='full')
        temp['cluster_' + expt] = dpgmm.fit_predict(temp[['index', expt]])

        gmm = mixture.GaussianMixture(n_components=len(set(temp['cluster_' + expt])), covariance_type='full').fit(temp[['index', expt]])
        x_min = normalized_df['index'].min()
        x_max = normalized_df['index'].max()

        x_range = np.linspace(x_min, x_max, len(normalized_df))
        y_range = np.linspace(normalized_df[expt].min(), normalized_df[expt].max(), len(normalized_df))
        X, Y = np.meshgrid(x_range, y_range)

        pdf = np.exp(gmm._estimate_log_prob(np.column_stack([X.ravel(), Y.ravel()])))
        overall_pdf = np.sum(pdf, axis=1)
        
        axs[i].plot(x_range, overall_pdf[0:len(x_range)], linestyle='--', linewidth=2)
        axs[i].plot(x_range, pdf[0:len(x_range)])
        axs[i].set_title(title_label[i], fontsize=fsz-8)
        axs[i].set_ylim(bottom=0, top=0.3)
        
        axs[i].tick_params('both',which='major', length=7,labelsize=fsz-8)
        axs[i].tick_params('both',which='minor', length=7,labelsize=fsz-8)
        i = i + 1

    # Set common labels
    fig.text(0.5, 0.04, 'Genomic coordinates sorted by their position', ha='center', fontsize=fsz-10, color='k')
    fig.text(0.04, 0.5, 'Probability distribution of gene expression', va='center', rotation='vertical', fontsize=fsz-10, color='k')

    plt.tight_layout(rect=[0.05, 0.05, 1, 0.95])
    # plt.savefig(os.path.join(output_folder,("P_dist_genomic_coord_chrom6_gamma1"+".pdf")), bbox_inches='tight')
    plt.show()

In [ ]:
df = pd.read_csv(input_file)
for chromosome,data in df.groupby('Chromosome'):
    expt_list = [item for sublist in rep_list for item in sublist]
    if (chromosome.isdigit() and int(chromosome) == 6):
        data.sort_values(by=['StartPos','EndPos'],inplace=True) 
        normalized_df = data_preprocessing(data,expt_list)
        normalized_df = normalized_df.drop(['index'],axis=1)
        normalized_df = normalized_df.reset_index()
        break

In [ ]:
mask_6 = pd.read_csv(os.path.join(ec_hc_mask_folder, "chromosome_6", 'chromosome_6_ec_hc_mask_xy.csv'))
expt_name_WT = ['WT_1_1','WT_2_1','WT_3_1']
expt_name_NT = ['HGPS_917_NT_1', 'HGPS_478_NT_1', 'HGPS_297_NT_1']
expt_name_comp = [expt_name_WT[0],expt_name_NT[0]]
fit_distribution_2d_comp(normalized_df,expt_name_comp)